You are an expert Python developer specializing in the Databricks environment. Your task is to create a complete Python script to be executed within a Databricks notebook. The script must perform the following operations:
1.	Data Retrieval from SpaceX API:
o	Interact with the SpaceX v3 REST API (https://api.spacexdata.com/v3).
o	Retrieve data from two specific endpoints: 
	All launches: https://api.spacexdata.com/v3/launches
	All rockets: https://api.spacexdata.com/v3/rockets
o	Handle potential errors during the API calls (e.g., timeouts, non-200 status codes).
2.	JOIN Operation:
o	Perform a "JOIN" operation to combine information from the launches data and the rockets data.
o	Join Logic: For each launch record, look up the corresponding rocket record using the rocket identifier. The join key is the rocket.rocket_id field within each launch record, which matches the rocket_id field in the rocket records. Add the rocket_name from the matched rocket record to the launch record. (This is conceptually similar to a left join, keeping all launch records).
o	The final result should be a list of dictionaries, where each dictionary represents a launch enriched with the corresponding rocket_name.
3.	Control Parameters and Debugging:
o	Include variables at the beginning of the script to define the API endpoint URLs, making them easily modifiable: 
	LAUNCHES_API_URL = "https://api.spacexdata.com/v3/launches"
	ROCKETS_API_URL = "https://api.spacexdata.com/v3/rockets"
o	Use Python's standard logging module to provide informative output during execution. Configure logging to display messages at the INFO level.
o	Log key messages such as: starting data retrieval (for launches, for rockets), number of launches retrieved, number of rockets retrieved, starting join operation, join operation complete, number of records after join, starting upload to httpbin, upload outcome.
4.	Execution Time Measurement:
o	Code Execution Time: Measure the time taken to perform the main operations (data retrieval + join operation). Print this time after the join operation is complete.
o	Pipeline Execution Time: Measure the total execution time of the entire script (from the beginning until after the upload to httpbin). Print this total time at the end of the script. Use Python's time module.
5.	Upload Result:
o	Take the resulting joined list of dictionaries from the join operation.
o	Serialize it into JSON format.
o	Make an HTTP POST request to the https://httpbin.org/post endpoint, sending the resulting joined JSON data in the request body.
o	Verify the response from httpbin.org (e.g., check the status code) and log the outcome of the upload operation.


In [0]:
import requests
import logging
import json
import time

# Step 1: Define API endpoint URLs
LAUNCHES_API_URL = "https://api.spacexdata.com/v3/launches"
ROCKETS_API_URL = "https://api.spacexdata.com/v3/rockets"

# Step 2: Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Step 3: Measure total pipeline execution time
start_pipeline_time = time.time()

def fetch_data(api_url):
    """
    Fetch data from the given API URL.
    Handles potential errors such as timeouts or non-200 status codes.
    """
    try:
        logging.info(f"Starting data retrieval from {api_url}")
        response = requests.get(api_url, timeout=10)
        response.raise_for_status()  # Raise an exception for HTTP errors
        data = response.json()
        logging.info(f"Retrieved {len(data)} records from {api_url}")
        return data
    except requests.exceptions.RequestException as e:
        logging.error(f"Error fetching data from {api_url}: {e}")
        return None

def join_launches_with_rockets(launches, rockets):
    """
    Perform a left join of launches with rockets using rocket_id.
    Add rocket_name to each launch record.
    """
    logging.info("Starting join operation")
    start_join_time = time.time()

    # Create a dictionary for quick lookup of rockets by rocket_id
    rockets_dict = {rocket['rocket_id']: rocket['rocket_name'] for rocket in rockets}

    # Enrich each launch record with the corresponding rocket_name
    enriched_launches = []
    for launch in launches:
        rocket_id = launch.get('rocket', {}).get('rocket_id')
        rocket_name = rockets_dict.get(rocket_id, None)  # Default to None if no match found
        enriched_launch = launch.copy()
        enriched_launch['rocket_name'] = rocket_name
        enriched_launches.append(enriched_launch)

    end_join_time = time.time()
    logging.info(f"Join operation complete. Time taken: {end_join_time - start_join_time:.2f} seconds")
    logging.info(f"Number of records after join: {len(enriched_launches)}")
    return enriched_launches

def upload_to_httpbin(data):
    """
    Serialize the data into JSON and upload it to https://httpbin.org/post.
    Log the outcome of the upload operation.
    """
    logging.info("Starting upload to httpbin")
    url = "https://httpbin.org/post"
    headers = {'Content-Type': 'application/json'}
    try:
        response = requests.post(url, data=json.dumps(data), headers=headers, timeout=10)
        response.raise_for_status()
        logging.info(f"Upload successful. Response status code: {response.status_code}")
        logging.debug(f"Response body: {response.text}")
    except requests.exceptions.RequestException as e:
        logging.error(f"Error uploading to httpbin: {e}")

def main():
    # Step 4: Measure time for data retrieval and join operations
    start_main_operations_time = time.time()

    # Retrieve data from SpaceX API
    launches = fetch_data(LAUNCHES_API_URL)
    rockets = fetch_data(ROCKETS_API_URL)

    if launches is None or rockets is None:
        logging.error("Data retrieval failed. Exiting script.")
        return

    # Perform the JOIN operation
    enriched_launches = join_launches_with_rockets(launches, rockets)

    end_main_operations_time = time.time()
    logging.info(f"Time taken for data retrieval and join operations: {end_main_operations_time - start_main_operations_time:.2f} seconds")

    # Step 5: Upload the joined data to httpbin
    upload_to_httpbin(enriched_launches)

    # Step 6: Measure total pipeline execution time
    end_pipeline_time = time.time()
    logging.info(f"Total pipeline execution time: {end_pipeline_time - start_pipeline_time:.2f} seconds")

# Execute the main function
if __name__ == "__main__":
    main()